# Recipe Finder with GenAI

🇺🇸 This project allows user to give the desired ingredients, any dietary restrictions and/or preferred culinary and get a recipe selection in return. It also allows the user to ask for a drink combination for the selected recipe.

🇧🇷 Esse projeto permite que o usuário insira os ingredientes desejados, qualquer restrição alimentar e/ou culinária preferida e receba uma lista de receitas. Ele também permite que o usuário peça recomendações de bebidas que combinam com a receita selecionada.

## Step 1: install genai and adk SDKs

In [ ]:
%pip -q install google-genai

In [ ]:
%pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/

## Step 2: import dependencies and setup environment

In [43]:
import os
from google.colab import userdata
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
import textwrap
from IPython.display import display, Markdown
import requests
import warnings

warnings.filterwarnings("ignore")


os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

## Step 3: create auxiliar functions

🇺🇸 These two auxiliar functions are used to help us call an agent and display the response in a prettier format.

🇧🇷 Essas duas funções auxiliares são usadas para nos ajudar a chamar um agente e exibir a resposta.

In [50]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## Step 4: create the agents

🇺🇸 Four used agents:
1.   **Translator Agent:** translate the texts into the desired language.
2.   **Search Agent:** search for recipes within the user input.
3.   **Recipe Agent:** give the full recipe steps from the searched recipe list.
4.   **Drink Agent:** give a perfect pair drink recommendation based on the selected recipe.

🇧🇷 Quatro agentes usados:
1.   **Translator Agent:** traduz os textos na linguagem desejada.
2.   **Search Agent:** procura por receitas com base na entrada do usuário.
3.   **Recipe Agent:** da todos os passos da receita escolhida.
4.   **Drink Agent:** da uma recomendação de bebida perfeita com base na receita selecionada.


In [51]:
####################################
# --- Agent 1: Translate texts --- #
####################################
def translate_text(language, text):
    translator_agent = Agent(
        name="translator_agent",
        model=MODEL_ID,
        instruction="""
        You are a highly accurate translator agent.
        Your task is to translate the given text *exactly* into the specified language, preserving any formatting or special characters.
        Do not add any extra commentary or explanations.  Do not add extra new lines (\n). Just return the translated text.
        """,
        description="The agent translates texts with high accuracy.",
        tools=[]
    )

    message_text = f"Translate the following text into {language}: '{text}'"
    return call_agent(translator_agent, message_text).strip()


#######################################
# --- Agent 2: Search for Recipes --- #
#######################################
def search_recipes(language, ingredients, dietary_restrictions, culinary):
  search_agent = Agent(
      name="search_agent",
      model=MODEL_ID,
      instruction="""
      You are a research assistant. Your role is to use google_search to find recipes based on ingredients,
      dietary restrictions and culinary type.
      Get a max of 5 relevant recipes based on the requirements.
      Please respond in the selected language.
      """,
      description="The agent searches for recipes on Google.",
      tools=[google_search],
  )

  search_agent_entry = f"Selected Language: {language}.\nIngredients: {ingredients}.\nDietary restrictions: {dietary_restrictions}.\nCulinary: {culinary}."

  return call_agent(search_agent, search_agent_entry)


#########################################
# --- Agent 3: Return chosen recipe --- #
#########################################
def give_recipe(language, chosen_recipe, recipe_list):
    recipe_agent = Agent(
        name="recipe_agent",
        model=MODEL_ID,
        instruction="""
        You are a recipe agent. Based on the recipe list, and the chosen recipe.
        You should use google_search to retrun the step by step recipe for the chosen recipe.
        If you can find, also give the estimated time it takes for cooking this recipe.
        Please respond in the selected language.
        """,
        description="The agent gives the steps to the chosen recipe.",
        tools=[google_search]
    )

    recipe_agent_entry = f"Selected language: {language}.\nChosen recipe:{chosen_recipe}.\nRecipe List: {recipe_list}"
    return call_agent(recipe_agent, recipe_agent_entry)


###############################################
# --- Agent 4: Give drink recommendations --- #
###############################################
def give_drink_recommendations(language, chosen_recipe, recipe_list):
    drink_agent = Agent(
        name="drink_agent",
        model=MODEL_ID,
        instruction="""
        You are a drink speciallist. Based on the recipe list, and the chosen recipe.
        You should use google_search to give a recommendation of a drink that would be a perfect match with the chosen recipe.
        Give a max of 3 drink options.
        Please respond in the selected language.
        """,
        description="The agent gives you a recomendation of drinks to match the recipe.",
        tools=[google_search]
    )

    drink_agent_entry = f"Selected language: {language}.\nChosen recipe:{chosen_recipe}.\nRecipe List: {recipe_list}"
    return call_agent(drink_agent, drink_agent_entry)

## Step 5: run the program

In [54]:
# Select language for the program
language = input("❓ Please select a language: ")
if not language:
  print("Defaulting to english...")
  language = "english"

# Choose ingredients, dietary restrictions and culinary
ingredients = input(translate_text(language, "❓ Please type the ingredients you would like to use:") + " ")
dietary_restrictions = input(translate_text(language, "❓ Please type any dietary restrictions:") + " ")
culinary = input(translate_text(language, "❓ If you have a preferred culinary type please state it here:") + " ")

print("\n")
if not ingredients:
  print(translate_text(language, "You need to at least give the ingredients!"))
else:
  print(translate_text(language, "Lets search for your recipe..."))

  # Seach for recipes
  recipe_list = search_recipes(language, ingredients, dietary_restrictions, culinary)
  print("\n--- " + translate_text(language, "Recipe List") + " ---\n")
  display(to_markdown(recipe_list))
  print("--------------------------------------------------------------\n")

  # Return chosen recipe
  chosen_recipe = input(translate_text(language, "❓ Please chose one of the above recipes to get more details:") + " ")
  while chosen_recipe.lower() != "done":
    recipe = give_recipe(language, chosen_recipe, recipe_list)
    print("\n--- " + translate_text(language, "Chosen Recipe") + " ---\n")
    display(to_markdown(recipe))
    print("--------------------------------------------------------------\n")
    chosen_recipe = input(translate_text(language, "❓ Would you like to get the details for another recipe? Please type the name here or write")
                          + " 'done' " + translate_text(language, "to move to the next question:") + " ")

  # Drinks that may be good with the chosen recipe
  drink_recommendation = (input(translate_text(language, "❓ Would you like some drink recommendation to match some of the recipes? Type")
                            + " 'y' " + translate_text(language, "for 'yes' and") + " 'n' " +
                            translate_text(language, "for 'no':") + " "))
  if drink_recommendation.lower() == "y":
    selected_recipe = input(translate_text(language, "❓ For which recipe would you like the drink recommendation?") + " ")
    drink = give_drink_recommendations(language, selected_recipe, recipe_list)
    print("\n--- " + translate_text(language, "Drink Recommendation") + " ---\n")
    display(to_markdown(drink))
    print("--------------------------------------------------------------\n")


❓ Please select a language: en
❓ Please type the ingredients you would like to use: banana, chocolate, oats
❓ Please type any dietary restrictions: lactose
❓ If you have a preferred culinary type please state it here: 


Lets search for your recipe...

--- Recipe List ---



> Here are some lactose-free recipes featuring banana, chocolate, and oats:
> 
> 1.  **Banana Chocolate Chunk Baked Oatmeal:** This recipe uses cashew milk to ensure it is dairy-free. It combines oats, bananas, cashew milk, eggs, vanilla extract, coconut oil, cinnamon, salt, baking soda, and chocolate chunks. It is baked for 45 minutes at 350°F.
> 2.  **Chocolate Banana Oat Breakfast Smoothie:** This smoothie uses non-dairy milk and includes rolled oats, almond butter, cocoa powder, flax seeds, vanilla extract, banana, salt, cinnamon, and optional protein powder and ice. Blend until smooth.
> 3.  **Vegan Chocolate Banana Oatmeal:** This recipe uses cashew milk, quick-cooking oats, cocoa powder, salt, cinnamon, maple syrup, vegan dark chocolate chips, and banana. Simmer for about 5 minutes.
> 4.  **Double Chocolate Banana Baked Oats:** This recipe calls for ripe bananas, dairy-free chocolate chips, cocoa powder, sugar or maple syrup, rolled oats, and dairy-free milk. Bake for 35-40 minutes at 350°F.
> 5.  **Banana Chocolate Overnight Oats:** This recipe uses almond milk to keep it vegan and lactose-free. It combines banana, almond milk, maple syrup, peanut butter, vanilla extract, oats, cocoa powder, chia seeds, cinnamon, salt, and optional chocolate chips.


--------------------------------------------------------------

❓ Please chose one of the above recipes to get more details: Banana Chocolate Overnight Oats

--- Chosen Recipe ---



> Here's a recipe for Banana Chocolate Overnight Oats, combining the information from multiple sources:
> 
> **Ingredients:**
> 
> *   1 large ripe banana, mashed.
> *   1/2 cup rolled oats.
> *   1 1/4 cup almond milk (or milk of your choice).
> *   1 tablespoon chia seeds.
> *   1 tablespoon cacao powder.
> *   1 tablespoon maple syrup.
> *   1/2 teaspoon vanilla extract.
> *   1/8 teaspoon cinnamon.
> *   Pinch of salt.
> *   Optional: chocolate chips, toppings of your choice.
> 
> **Instructions:**
> 
> 1.  In a medium-sized bowl, mash the banana.
> 2.  Add the cacao powder, maple syrup, and vanilla to the mashed banana. Mix until well combined.
> 3.  Add the milk, oats, chia seeds, and cinnamon. Stir to combine.
> 4.  Cover the bowl with cling wrap, or divide the mixture into a glass or mason jar and cover.
> 5.  Refrigerate for at least 6 hours, or preferably overnight.
> 6.  In the morning, stir the oats and add more milk if needed to reach your desired consistency.
> 7.  Serve with toppings of your choice.
> 
> **Estimated Time:**
> 
> *   Preparation: 5-10 minutes.
> *   Refrigeration: 6 hours or overnight.
> 


--------------------------------------------------------------

❓ Would you like to get the details for another recipe? Please type the name here or write' 'done' to move to the next question: 4

--- Chosen Recipe ---



> Here's how to make Double Chocolate Banana Baked Oats, with an estimated baking time of 35-40 minutes:
> 
> **Ingredients:**
> 
> *   2 medium ripe bananas
> *   1/2 cup (84 grams) chocolate chips, plus extra for topping
> *   1/2 cup (40 grams) cocoa powder
> *   2 tablespoons sugar or maple syrup
> *   2 cups (200 grams) rolled/old-fashioned oats
> *   1 cup (240 ml) dairy-free milk
> *   Optional: a small handful of chopped walnuts or pecans
> 
> **Instructions:**
> 
> 1.  Preheat your oven to 350°F (175°C).
> 2.  In a large bowl, mash the bananas until smooth. You can even do this in the oven-proof dish to save on washing up.
> 3.  Add the chocolate chips, cocoa powder, maple syrup/sugar, oats, and dairy-free milk to the bananas and stir to combine.
> 4.  Spoon the mixture into an oven-proof dish and smooth the top. Sprinkle with additional chocolate chips and nuts if desired.
> 5.  Bake for 35 to 40 minutes, or until heated through and a bit crusty on top.
> 6.  Serve warm or cold.
> 


--------------------------------------------------------------

❓ Would you like to get the details for another recipe? Please type the name here or write' 'done' to move to the next question: done
❓ Would you like some drink recommendation to match some of the recipes? Type 'y' for 'yes' and 'n' for 'no':' y
❓ For which recipe would you like the drink recommendation? Banana Chocolate Overnight Oats

--- Drink Recommendation ---



> Here are a few drink recommendations that would pair well with your Banana Chocolate Overnight Oats:
> 
> 1.  **Chocolate Banana Smoothie:** Given the flavors in your overnight oats, a chocolate banana smoothie would complement it nicely. You can adjust the ingredients to match your preferences, such as using almond milk or adding protein powder.
> 2.  **Milk:** A simple glass of milk (dairy or non-dairy) can be a refreshing and classic pairing with oatmeal. The cool and plain flavor of the milk can balance the richness of the chocolate and banana.
> 3.  **Coffee:** For those who enjoy a caffeine boost in the morning, coffee can be a good choice. The bitterness of the coffee can contrast the sweetness of the banana and chocolate, creating a balanced combination.
> 


--------------------------------------------------------------

